# 整数规划
- 整数规划的模型与线性规划基本相同，只是额外增加了部分或所有决策变量为整数的约束

从决策变量（问题中要确定的未知量）的取值范围来看，整数规划通常可以分为以下三种类型：
- 纯整数规划： 全部决策变量都取整数
- 混合整数规划： 决策变量中有一部分必须取整数，另一些可以不取整数
- 0-1整数规划： 决策变量只能取0或1的整数规划

对于0-1整数规划：约束条件加上 xi（1-xi）=0  i=1,2,3....  即可表示

## 1.分支定界法：

In [1]:
import math
from scipy.optimize import linprog
import sys

def integerPro(c, A, b, Aeq, beq,t=1.0E-8):
    res = linprog(c, A_ub=A, b_ub=b, A_eq=Aeq, b_eq=beq)
    bestVal = sys.maxsize
    bestX = res.x
    if not(type(res.x) is float or res.status != 0): 
        bestVal = sum([x*y for x,y in zip(c, bestX)])
    if all(((x-math.floor(x))<=t or (math.ceil(x)-x)<=t) for x in bestX):
        return (bestVal,bestX)
    else:
        ind = [i for i, x in enumerate(bestX) if (x-math.floor(x))>t and (math.ceil(x)-x)>t][0]
        newCon1 = [0]*len(A[0])
        newCon2 = [0]*len(A[0])
        newCon1[ind] = -1
        newCon2[ind] = 1
        newA1 = A.copy()
        newA2 = A.copy()
        newA1.append(newCon1)
        newA2.append(newCon2)
        newB1 = b.copy()
        newB2 = b.copy()
        newB1.append(-math.ceil(bestX[ind]))
        newB2.append(math.floor(bestX[ind]))
        r1 = integerPro(c, newA1, newB1, Aeq, beq)
        r2 = integerPro(c, newA2, newB2, Aeq, beq)
        if r1[0] < r2[0]:
            return r1
        else:
            return r2

## 2.pulp库

在变量设置中 设置 cat='Integer'

其余与1.线性规划部分差别不大

In [ ]:
 var = [[pulp.LpVariable(f'x{i}{j}',lowBound=0,cat=pulp.LpInteger) for j in range(col)]for i in range(row)]